In [1]:
!pip install gensim

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!pip install python-Levenshtein

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 50 kB 2.5 MB/s 
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.2-cp37-cp37m-linux_x86_64.whl size=149868 sha256=12871c450535f3a36ad224d03e0fc75875bcc9c1d3c5d0e66894724a0ea9727f
  Stored in directory: /root/.cache/pip/wheels/05/5f/ca/7c4367734892581bb5ff896f15027a932c551080b2abd3e00d
Successfully built python-Levenshtein


In [3]:
import gensim
import pandas as pd
import numpy as np

In [12]:
df = pd.read_csv("/content/plot_keywords_only.csv")

In [13]:
df.head()

,plot_keywords
0,huguenot|intolerance|medicis|protestant|wedding
1,family relationships|gang|idler|poorhouse|thief
2,chewing gum|climbing a tree|france|translation...
3,art deco|bible quote|dance|silent film|worker
4,escape|femme fatale|german expressionism|lust|...


In [14]:
df.plot_keywords[0]

'huguenot|intolerance|medicis|protestant|wedding'

In [15]:
gensim.utils.simple_preprocess("huguenot|intolerance|medicis|protestant|wedding")

['huguenot', 'intolerance', 'medicis', 'protestant', 'wedding']

In [16]:
plot_keywords = df.plot_keywords.apply(gensim.utils.simple_preprocess)
plot_keywords

0      [huguenot, intolerance, medicis, protestant, w...
1      [family, relationships, gang, idler, poorhouse...
2      [chewing, gum, climbing, tree, france, transla...
3      [art, deco, bible, quote, dance, silent, film,...
4      [escape, femme, fatale, german, expressionism,...
                             ...                        
595                       [student, film, wonder, years]
596    [apartment, dancing, debutante, new, york, upp...
597    [cigarette, smoking, death, devil, nightmare, ...
598    [assassin, death, female, protagonist, rogue, ...
599    [cat, burglar, female, lead, future, mixed, ma...
Name: plot_keywords, Length: 600, dtype: object

In [71]:
model = gensim.models.Word2Vec(
    window = 3,
    min_count = 2,
    workers = 4
)

In [72]:
model.build_vocab(plot_keywords, progress_per = 600)

In [73]:
model.epochs 

5

In [74]:
model.corpus_count

600

In [75]:
model.train(plot_keywords,total_examples=model.corpus_count,epochs=model.epochs )

(14756, 22060)

In [76]:
model.save("/content/model")

In [77]:
model.wv.most_similar("war")

[('ape', 0.2911757230758667),
 ('computer', 0.2883511185646057),
 ('parody', 0.2865104675292969),
 ('jungle', 0.27357757091522217),
 ('voodoo', 0.27174481749534607),
 ('sister', 0.26615744829177856),
 ('outer', 0.25260621309280396),
 ('niece', 0.2502369284629822),
 ('power', 0.24746522307395935),
 ('surgery', 0.24741864204406738)]

In [80]:
model.wv.similarity(w1="astronaut",w2="space")

0.10802555

In [82]:
vocab, vectors = model.wv.vocab, model.wv.vectors

# get node name and embedding vector index.
name_index = np.array([(v[0], v[1].index) for v in vocab.items()])

# init dataframe using embedding vectors and set index as node name
df =  pd.DataFrame(vectors[name_index[:,1].astype(int)])
df.index = name_index[:, 0]
df.to_csv("plot_keywords_embedding.csv")

In [85]:
# Save model
filename = 'keywords_model.wv' # Can be any arbitrary filename
model.save(filename) 

# Reload model
retrieved_model = gensim.models.Word2Vec.load(filename)